In [1]:
import re
import pandas as pd
from pathlib import Path
import requests
from bs4 import BeautifulSoup
import seaborn as sns
import numpy as np
from getpass import getpass

In [3]:
persons = pd.read_json('https://osat-api.herokuapp.com/persons').values

In [10]:
ids = [pp[0]['id'] for pp in persons]

In [17]:
ids[0]
person_url_base = 'https://osat-api.herokuapp.com/person/'


In [38]:
pmids = []
dois = []
for pid in ids:
    res = requests.get(person_url_base + pid)
    pmids.extend([pa['pmid'] for pa in res.json()['papers']])
    dois.extend([pa['metadata']['doi'] for pa in res.json()['papers']])
pmids = np.unique(pmids).astype(int)
dois = np.unique(dois)

TypeError: '<' not supported between instances of 'str' and 'NoneType'

In [37]:
res.json()['papers']

[{'metadata': {'abstract': 'Recent "representational" accounts suggest a key role for the hippocampus in complex scene perception. Due to limitations in scanner field strength, however, the functional neuroanatomy of hippocampal-dependent scene perception is unknown. Here, we applied 7 T high-resolution functional magnetic resonance imaging (fMRI) alongside a perceptual oddity task, modified from nonhuman primate studies. This task requires subjects to discriminate highly similar scenes, faces, or objects from multiple viewpoints, and has revealed selective impairments during scene discrimination following hippocampal lesions. Region-of-interest analyses identified a preferential response in the subiculum subfield of the hippocampus during scene, but not face or object, discriminations. Notably, this effect was in the anteromedial subiculum and was not modulated by whether scenes were subsequently remembered or forgotten. These results highlight the value of ultra-high-field fMRI in ge

In [34]:
pmids = np.unique(pmids)

In [35]:
with open('../data/osdash_pmids.txt', 'w') as h:
    pmids.tofile(h, sep='\n', format="%d")

In [45]:
apikey=getpass()
scop_jsons = []
for ii, pmid in enumerate(pmids):
    scopus_search_url = f'https://api.elsevier.com/content/search/scopus?query=PMID({pmid})&apiKey={apikey}'
    with requests.get(scopus_search_url) as res:
        scop_jsons.append(res.json())
    if ii % 10 == 0:
        print(ii, end=', ', flush=True)

0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390, 400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520, 530, 540, 550, 560, 570, 580, 590, 600, 610, 620, 630, 640, 650, 660, 670, 680, 690, 700, 710, 720, 730, 740, 750, 760, 770, 780, 790, 800, 810, 820, 830, 840, 850, 860, 870, 880, 890, 900, 910, 920, 930, 940, 950, 960, 970, 980, 990, 1000, 1010, 1020, 1030, 1040, 1050, 1060, 1070, 1080, 1090, 1100, 1110, 1120, 1130, 1140, 1150, 1160, 1170, 1180, 1190, 1200, 1210, 1220, 1230, 1240, 1250, 1260, 1270, 

In [76]:
scop_jsons[0]

dict_keys(['search-results'])

In [65]:
(np.array([int(sj['search-results']['opensearch:itemsPerPage']) for sj in scop_jsons])==1).sum()

1117

In [78]:
scop_jsons[0]['search-results']['entry'][0]

{'@_fa': 'true',
 'link': [{'@_fa': 'true',
   '@ref': 'self',
   '@href': 'https://api.elsevier.com/content/abstract/scopus_id/84856102195'},
  {'@_fa': 'true',
   '@ref': 'author-affiliation',
   '@href': 'https://api.elsevier.com/content/abstract/scopus_id/84856102195?field=author,affiliation'},
  {'@_fa': 'true',
   '@ref': 'scopus',
   '@href': 'https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=84856102195&origin=inward'},
  {'@_fa': 'true',
   '@ref': 'scopus-citedby',
   '@href': 'https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=84856102195&origin=inward'},
  {'@_fa': 'true',
   '@ref': 'full-text',
   '@href': 'https://api.elsevier.com/content/article/eid/1-s2.0-S0028390811000256'}],
 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/84856102195',
 'dc:identifier': 'SCOPUS_ID:84856102195',
 'eid': '2-s2.0-84856102195',
 'dc:title': 'GABAergic interneuron origin of schizophrenia pathophysiology',
 'dc:creator': 'Nakazawa K.',
 'prism

In [81]:
sjks = ['@_fa', 'prism:url', 'dc:identifier', 'eid', 'dc:title',
        'dc:creator', 'prism:publicationName', 'prism:issn', 'prism:eIssn',
        'prism:volume', 'prism:issueIdentifier', 'prism:pageRange', 
        'prism:coverDate', 'prism:coverDisplayDate', 'prism:doi', 
        'pii', 'citedby-count', 'pubmed-id', 
        'prism:aggregationType', 'subtype', 'subtypeDescription', 
        'source-id', 'openaccess', 'openaccessFlag']
scop_df = []
for sj in scop_jsons:
    sj_dict = {}
    for sjk in sjks:
        try:
            sj_dict[sjk] = sj['search-results']['entry'][0][sjk]
        except KeyError:
            pass
    scop_df.append(sj_dict)   
    
    

In [87]:
scop_df = pd.DataFrame(scop_df)
scop_df.to_csv('../data/scopus_cit_counts.csv', index=False)

In [92]:
scop_df.loc[pd.notnull(scop_df['citedby-count']), 'citedby-count'] = scop_df.loc[pd.notnull(scop_df['citedby-count']), 'citedby-count'].astype(int)

In [94]:
scop_df.sort_values('citedby-count', ascending=False).to_csv('../data/scopus_cit_counts.csv', index=False)


········


In [44]:
scop_jsons

[{'search-results': {'opensearch:totalResults': '0',
   'opensearch:startIndex': '0',
   'opensearch:itemsPerPage': '0',
   'opensearch:Query': {'@role': 'request',
    '@searchTerms': 'PMID(5684567865835673456745)',
    '@startPage': '0'},
   'link': [{'@_fa': 'true',
     '@ref': 'self',
     '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=PMID%285684567865835673456745%29&apiKey=7f59af901d2d86f78a1fd60c1bf9426a',
     '@type': 'application/json'}],
   'entry': [{'@_fa': 'true', 'error': 'Result set was empty'}]}}]